# mp4 파일 다운로드

In [2]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ip = '203.255.56.63'
port = 44444
sock.connect((ip,port))

length = int(sock.recv(1024).decode())
n_d = int(sock.recv(1024).decode())

open('/Users/winlab/Desktop/r_video.mp4','wb').write(sock.recv(length))
for i in range(n_d - 1):
    open('/Users/winlab/Desktop/r_video.mp4','ab').write(sock.recv(length))

sock.close()

# mp4  파일 다운로드 + 인덱싱 하면서 재생

In [1]:
import socket
import numpy as np
import cv2
import threading
import time

def indexing_mp4(v1_data, v2_data, count):
    np_v1 = np.frombuffer(v1_data, dtype='uint8') # 바이너리 해석
    np_v2 = np.frombuffer(v2_data[count:count+length], dtype='uint8')

    xor_len = min(len(np_v1), len(np_v2))
    
    xor_v_data = np.bitwise_xor(np_v1[:xor_len], np_v2[:xor_len]) # xor 연산
    
    xor_v_bin = xor_v_data.tostring() # 택 데이터 바이너리화
    return xor_v_bin

original_path = '/Users/winlab/Desktop/Summer.mp4'  # 가지고 있는 동영상의 경로
stream_path = '/Users/winlab/Desktop/stream1.mp4'  # 저장할 경로와 이름

with open(original_path, 'rb') as v2: # 가지고 있는 영상 바이너리 형식으로 읽어오기
    v2_data = v2.read()
    
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ip = '203.255.56.63'
port = 44444
sock.connect((ip,port))

sock.send('Summer'.encode())  # 가지고 있는 동영상 이름
length = int(sock.recv(1024).decode())
n_d = int(sock.recv(1024).decode())  # 데이터 분열 갯수

header_data = b''
count = 0
    
open(stream_path, 'wb').write(indexing_mp4(sock.recv(length), v2_data, count))
count += length

def recv():  # 데이터 받아서 인덱싱
    global count
    for i in range(n_d - 1):
        v1_data = sock.recv(length)
        data = indexing_mp4(v1_data, v2_data, count)
        open(stream_path, 'ab').write(data)
        count += length
        if count >= len(v2_data):
            open(stream_path, 'ab').write(v1_data[len(data):])
            break
    for i in range(i+1, n_d - 1):
        open(stream_path, 'ab').write(sock.recv(length))

t = threading.Thread(target = recv)
t.start()

index = 0
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

while 1:
    video = cv2.VideoCapture(stream_path)
    
    if video.get(5) == 0:  # 받은 데이터가 없다면 잠시 쉬었다가 다시 실행
        time.sleep(0.1)
        continue
        
    f = round(1/video.get(5)*1000) - 7  # 초탕 프레임으로 waitkey값 계산
    
    video.set(1, index)  # 프레임 위치 설정
    while 1:
        ret, frame = video.read()
        if ret:
            cv2.imshow('Video', frame)
            cv2.waitKey(f)
        else:
            index = video.get(1)  # 프레임 위치 저장
            break
            
    if index != video.get(7):  # 총 프레임수 만큼 재생했다면 종료
        time.sleep(1)
        pass
    else:
        break
    
    video.release()

cv2.destroyAllWindows()
sock.close()